In [1]:
%reset
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne
from mne import create_info
import pickle

## Import epochs

In [ ]:
epochs_dir = 'C:\AnatArzData\YKM_data\epochs_and_evoked_allSubs'
import_type = "5Electorodes_plainEEGLAB"
fig_output_dir = "C:\\Users\Sharon\OneDrive\Desktop\AnatArzi\scripts\Python_scripts - Sharons\mycode\outputs\\erps"

In [ ]:
with open(f'{epochs_dir}\\{import_type}.pkl', 'rb') as file:
    [allEvents_df, allEpochs_perCond, cfg] = pickle.load(file)

times = cfg['times']

In [ ]:
allEvents_df

In [ ]:
def getEpochsPerCond(cond_df):
    curr_df =cond_df.copy()
    df_minTrials = curr_df[(curr_df.SamplesAmount > 0)] # discard cond with 0 enough samples
    minTrials_cond_keys = np.asarray([str (i) for i in df_minTrials['Cond_id']])
    epochs_allSamples = {}
    for key in minTrials_cond_keys:
        epochs_allSamples[key] = allEpochs_perCond[key]
    return df_minTrials, epochs_allSamples

# output: [#conds, #elect, #times]
def getEvokedPerCondAndElectd(constraints,type='array'):
    curr_df =allEvents_df.copy()
    # apply constraints
    for key in constraints: curr_df = curr_df[(curr_df[key] == constraints[key])]

    conds_df, epochsPerCond = getEpochsPerCond(curr_df)
    evoked_perCond_andElectd = np.zeros((len(epochsPerCond),np.size(cfg['electrodes']),np.size(times)))

    for cond_i, cond in enumerate(epochsPerCond):
        evoked_perCond_andElectd[cond_i] = np.squeeze(np.nanmean(epochsPerCond[cond],axis=2))

    if type =='array':
        return conds_df, evoked_perCond_andElectd
    if type =='mne':
        info=create_info(ch_names=cfg['ch_names'],ch_types='eeg',sfreq=250)
        mne_epochs = mne.EpochsArray(evoked_perCond_andElectd, info,tmin=-0.1)
        return conds_df, mne_epochs

## Plot thickens

In [ ]:
def applyDesign(ax,title='',y_lims=[-4,4]):
    ax.set_facecolor('silver')
    ax.set_title(title,fontsize = 18, fontweight = 'bold')
    ax.legend(loc = 'upper right',prop={'size': 10})
    ax.axvline(x=0,color='gray', linestyle='--',label ="_nolegend_")
    ax.axhline(y=0, color='gray', linestyle='-',label ="_nolegend_")
    ax.set_ylabel(' uV', fontsize = 16, fontweight = 'bold')
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_xlabel('Time (ms)', fontsize = 16, fontweight = 'bold')
    ax.set_xlim([-100,450])
    ax.set_ylim(y_lims)
    ax.text(-0.02, y_lims[1], 'stimulus', rotation=90, va='center', fontsize = 13)

colors_wngt = plt.cm.Blues([250,230,210,190,170,150,130,110,90])
colors_N2 = plt.cm.autumn_r([150,140,130,120,110,100,80,60,30])
colors_N3 = plt.cm.Greens([250,230,210,190,170,150,130,110,90])
colors_REM = plt.cm.Reds([250,230,210,190,170,150,130,110,90])
colors_allShadesPerVg = [colors_wngt,colors_N2,colors_N3,colors_REM]
colors_onePerVg = [colors_wngt[2],colors_N2[2],colors_N3[2],colors_REM[2]]
titles_onePerVg = ['Awake','N2','N3','REM']

In [ ]:
def plot_erps_with_std(varNameForMean, varNameForStd,constraints,ax, colors,labels,varsForMean=None,varsForStd=None, linestyle='-'):
    if varsForMean is None: varsForMean =  allEvents_df[varNameForMean].unique()
    for meanVar_i,meanVar in enumerate(varsForMean):
        if varNameForStd == 'Cond_id':
            curr_constraints = constraints.copy()
            curr_constraints[varNameForMean] = meanVar
            conds_df, evoked_mat = getEvokedPerCondAndElectd(curr_constraints,type='array')
            evokedPerStdVar = np.mean(evoked_mat,axis=1) # mean over electrodes
            erp_trialsNum = conds_df['SamplesAmount'].sum()
        else:
            if varsForStd is None: varsForStd =  allEvents_df[varNameForStd].unique()
            evokedPerStdVar = np.zeros((len(varsForStd), len(times)))
            erp_trialsNum=0
            for stdVar_i, stdVar in enumerate(varsForStd):
                curr_constraints = constraints.copy()
                curr_constraints[varNameForStd] = stdVar
                curr_constraints[varNameForMean] = meanVar

                conds_df, evoked_mat = getEvokedPerCondAndElectd(curr_constraints,type='array')
                evokedPerStdVar[stdVar_i,:] = np.nanmean(evoked_mat,axis=(0,1))
                erp_trialsNum += conds_df['SamplesAmount'].sum()

        curr_meanEvoked = np.mean(evokedPerStdVar,axis=0)
        curr_stdEvoked = np.std(evokedPerStdVar,axis=0)
        ax.fill_between(times,curr_meanEvoked+curr_stdEvoked,curr_meanEvoked-curr_stdEvoked, color=colors[meanVar_i],alpha=0.1,label ="_nolegend_")
        label = f"{labels[meanVar_i]}, n={erp_trialsNum}"
        ax.plot(times,curr_meanEvoked, color=colors[meanVar_i],label=label,linestyle=linestyle)
    return ax

In [ ]:
def plot_erps(varNameForMean, constraints,ax, colors,labels):
    varForMean =  allEvents_df[varNameForMean].unique()
    for vg_i,vg in enumerate(varForMean):
        curr_constraints = constraints.copy()
        curr_constraints[varNameForMean] = vg

        conds_df, evoked_mat = getEvokedPerCondAndElectd(curr_constraints,type='array')
        curr_meanEvoked = np.nanmean(evoked_mat,axis=(0,1))
        ax.plot(times,curr_meanEvoked,color=colors[vg_i],label=labels[vg_i])
    return ax

Only in "Tone" condition. Each graph contatins ERP mean for each vigi-state

In [ ]:
fig, ax = plt.subplots()
plot_erps('Vigilance', {'TOA_cond':'T'}, ax,colors_onePerVg,titles_onePerVg)
plot_title = 'Only T, all subs'
applyDesign(ax,plot_title)
plt.ioff()
# plt.show()

fig, ax = plt.subplots()
plot_erps_with_std('Vigilance','Subject', {'TOA_cond':'T'}, ax,colors_onePerVg,titles_onePerVg)
plot_title = 'Only T, all subs'
applyDesign(ax,plot_title)
plt.ioff()
plt.savefig(f'{fig_output_dir}/T_allSubs_5CentElect.png')
# plt.show()

## Per sub, Only T and only O

In [ ]:
for sub in cfg['subs']:
    fig, axs = plt.subplots(1, 3,sharex=True, sharey=False,figsize = (20,6))

    constraints = {'TOA_cond':'T','Subject':sub}
    plot_erps_with_std('Vigilance','Cond_id', constraints, axs[0],colors_onePerVg,titles_onePerVg)
    plot_title = f'Tone, sub {sub}'
    applyDesign(axs[0],plot_title,y_lims=[-6,6])

    constraints = {'TOA_cond':'Fixed','Subject':sub}
    plot_erps_with_std('Vigilance','Cond_id', constraints, axs[1],colors_onePerVg,titles_onePerVg)
    plot_title = f'Fixed O, sub {sub}'
    applyDesign(axs[1],plot_title,y_lims=[-6,6])

    constraints = {'TOA_cond':'Rand','Subject':sub}
    plot_erps_with_std('Vigilance','Cond_id', constraints, axs[2],colors_onePerVg,titles_onePerVg)
    plot_title = f'Rand O, sub {sub}'
    applyDesign(axs[2],plot_title,y_lims=[-6,6])

    plt.tight_layout()
    plt.savefig(f'{fig_output_dir}/TOA_sub{sub}_5CentElect.png')
    plt.show()


### Same, but without std

# for sub in cfg['subs']:
#     fig, axs = plt.subplots(1, 3,sharex=True, sharey=False,figsize = (13,3))
#
#     constraints = {'TOA_cond':'T','Subject':sub}
#     plot_erps('Vigilance', constraints, axs[0],colors_onePerVg,titles_onePerVg)
#     plot_title = f'Tone, sub {sub}'
#     applyDesign(axs[0],plot_title,y_lims=[-6,6])
#
#     constraints = {'TOA_cond':'Fixed','Subject':sub}
#     plot_erps('Vigilance', constraints, axs[1],colors_onePerVg,titles_onePerVg)
#     plot_title = f'Fixed O, sub {sub}'
#     applyDesign(axs[1],plot_title,y_lims=[-6,6])
#
#     constraints = {'TOA_cond':'Rand','Subject':sub}
#     plot_erps('Vigilance', constraints, axs[2],colors_onePerVg,titles_onePerVg)
#     plot_title = f'Rand O, sub {sub}'
#     applyDesign(axs[2],plot_title,y_lims=[-6,6])
#
#     plt.tight_layout()
#     plt.show()

In [ ]:
fig, axs = plt.subplots(2, len(cfg['vigi_states'])//2,sharex=True, sharey=True,figsize = (15,10))
axs = np.reshape(axs,[len(cfg['vigi_states'])])
for vg_i, vg in enumerate(cfg['vigi_states']):
    colors_pallte=colors_allShadesPerVg[vg_i]
    plot_erps_with_std('ToneHeight','Subject', {'TOA_cond':'T','Vigilance':vg}, axs[vg_i],colors_pallte,cfg['toneHeights'])
    plot_title = f"{titles_onePerVg[vg_i]}"
    applyDesign(axs[vg_i],plot_title)
plt.tight_layout()
plt.suptitle('T only, all valid subs', x = 0.5, y = 1.05, fontsize=20, weight='bold')
plt.ioff()
plt.savefig(f'{fig_output_dir}/T_perVigi_perToneH_allValidSubs_5CentElect.png')
plt.show()

## Omissions: Number of trials per vigi state

In [ ]:
df_omission = pd.DataFrame({}, columns=['Subject', 'Vigilance', 'SamplesAmount_rand','SamplesAmount_fixed', 'Cond_id_rand','Cond_id_fixed'])
df_partly_filtered_fixed =  allEvents_df[(allEvents_df.TOA_cond == 'Fixed')]
df_partly_filtered_rand =   allEvents_df[(allEvents_df.TOA_cond == 'Rand')]

for sub_i, sub in enumerate(cfg['subs']):
    for vs_i, vs in enumerate(cfg['vigi_states']):
        curr_sub_vs_df_fixed = df_partly_filtered_fixed[(df_partly_filtered_fixed.Vigilance == vs) & (df_partly_filtered_fixed.Subject == sub)]
        curr_sub_vs_df_rand = df_partly_filtered_rand[(df_partly_filtered_rand.Vigilance == vs) & (df_partly_filtered_rand.Subject == sub)]

        df_temp = pd.DataFrame({
                    "Subject":[sub],
                    "Vigilance":[vs],
                    'SamplesAmount_rand':[np.sum(curr_sub_vs_df_rand.SamplesAmount)],
                    'SamplesAmount_fixed':[np.sum(curr_sub_vs_df_fixed.SamplesAmount)],
                    'Cond_id_rand':[np.array(curr_sub_vs_df_rand.Cond_id)],
                    'Cond_id_fixed':[np.array(curr_sub_vs_df_fixed.Cond_id)]})
        df_omission = pd.concat((df_omission, df_temp),ignore_index=True)

df_lessThan30Trials = df_omission[(df_omission.SamplesAmount_rand < 30) | (df_omission.SamplesAmount_fixed < 30)]
df_lessThan30Trials.to_excel(f'{fig_output_dir}/df_omission_lessThan30Trials.xlsx')
df_omission.to_excel(f"{fig_output_dir}/df_omission_all.xlsx")
display(df_lessThan30Trials)
# df_omission_subs = df_omission[(df_omission.Subject == '12') | (df_omission.Subject == '17') |(df_omission.Subject == '34')]
# df_omission_subs

## Omissions vs Tone: Number of trials per vigi state

In [ ]:
df_omission_vs_tone = pd.DataFrame({}, columns=['Subject', 'Vigilance', 'SamplesAmount_omis','SamplesAmount_tone', 'SamplesAmount_ratio','Cond_id_omis','Cond_id_tone'])
df_partly_filtered_omis =  allEvents_df[(allEvents_df.TOA == 'O')]
df_partly_filtered_tone =  allEvents_df[(allEvents_df.TOA == 'T') ]

for sub_i, sub in enumerate(cfg['subs']):
    for vs_i, vs in enumerate(cfg['vigi_states']):
        curr_sub_vs_df_omis = df_partly_filtered_omis[(df_partly_filtered_omis.Vigilance == vs) & (df_partly_filtered_omis.Subject == sub)]
        curr_sub_vs_df_tone = df_partly_filtered_tone[(df_partly_filtered_tone.Vigilance == vs) & (df_partly_filtered_tone.Subject == sub)]


        df_temp = pd.DataFrame({
                    "Subject":[sub],
                    "Vigilance":[vs],
                    'SamplesAmount_omis':[np.sum(curr_sub_vs_df_omis.SamplesAmount)],
                    'SamplesAmount_tone':[np.sum(curr_sub_vs_df_tone.SamplesAmount)],
                    'SamplesAmount_ratio':[np.sum(curr_sub_vs_df_omis.SamplesAmount)/np.sum(curr_sub_vs_df_tone.SamplesAmount)],
                    'Cond_id_omis':[np.array(curr_sub_vs_df_omis.Cond_id)],
                    'Cond_id_tone':[np.array(curr_sub_vs_df_tone.Cond_id)]})
        df_omission_vs_tone = pd.concat((df_omission_vs_tone, df_temp),ignore_index=True)

df_omission_vs_tone.to_excel(f'{fig_output_dir}/df_omissionVStones_trialsRatio.xlsx')
df_omission_vs_tone

In [ ]:
## initial filter full dataframe
curr_df = allEvents_df[(allEvents_df.TOA == 'O') & (allEvents_df.TrialPosition > '4')]
curr_subs = curr_df.Subject.unique()
curr_vss = curr_df.Vigilance.unique()

conds_names = [['Fixed, 10th place'], ['Random, 7th place'],['Random, 8th place'],['Random, 9th place']]
conds_linestyles =['-', '--','--','--']

constraints = [{"TOA_cond": "Fixed"},
               {"TOA_cond": "Rand","TrialPosition" : "6"},
               {"TOA_cond": "Rand","TrialPosition" : "7"},
               {"TOA_cond": "Rand","TrialPosition" : "8"},]

fig, axs = plt.subplots(2, len(cfg['vigi_states'])//2,sharex=True, sharey=True,figsize = (15,10))
axs = np.reshape(axs,[len(cfg['vigi_states'])])
for vg_i, vg in enumerate(cfg['vigi_states']):
    colors_pallte=colors_allShadesPerVg[vg_i]
    for const_i ,constraint in enumerate(constraints):
        curr_constr = constraint.copy()
        curr_constr['Vigilance']=vg
        plot_erps_with_std("TOA_cond",'Subject', curr_constr, axs[vg_i],colors_allShadesPerVg[vg_i],conds_names[const_i], varsForMean=[constraint["TOA_cond"]],linestyle=conds_linestyles[const_i])

    plot_title = f"{titles_onePerVg[vg_i]}"
    applyDesign(axs[vg_i],plot_title)
plt.tight_layout()
plt.suptitle('O only, all valid subs', x = 0.5, y = 1.05, fontsize=20, weight='bold')

plt.ioff()
plt.savefig(f'{fig_output_dir}/Omis_perVigi_perTrialPos_allValidSubs_5CentElect.png')
plt.show()